# ADNI Preprocessing Pipeline with NiPype

In [4]:
import os
import shutil
import cv2 as cv
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

from tqdm import tqdm
from sklearn.model_selection import train_test_split

from nipype.interfaces.freesurfer import ReconAll
from nipype.interfaces.utility import IdentityInterface
from nipype.pipeline.engine import Workflow, Node

Set paths.

In [2]:
experiment_dir = "/Data/test"
data_dir = os.path.join(experiment_dir, "samples")
# out_dir = os.path.join(experiment_dir, "out")
# fs_folder = "/freesurfer"

fs_folder = os.path.join(experiment_dir, "freesurfer")
if not os.path.isdir(fs_folder):
    os.mkdir(fs_folder)

subject_list = ["ADNI_test_sample"]
T1_identifier = os.path.normpath("struct.nii.gz")

## Recon-all

In [3]:
reconflow = Workflow(name="reconflow")
reconflow.base_dir = os.path.join(data_dir, "workingdir_reconflow")

infosource = Node(IdentityInterface(fields=['subject_id']), name="inforsource")
infosource.iterables = ("subject_id", subject_list)

Define recon-all.

In [4]:
reconall = Node(ReconAll(
    directive="all", 
    subjects_dir=fs_folder), 
    name="reconall",
)

Define workflow.

In [5]:
def pathfinder(subject, foldername, filename):
    from os.path import join as pathjoin
    path =  pathjoin(foldername, subject, filename)
    return path

reconflow.connect([
    (infosource, reconall, [('subject_id', 'subject_id')]),
    (infosource, reconall, [(('subject_id', pathfinder, data_dir, T1_identifier), 'T1_files')]),
])

Run workflow.

In [6]:
reconflow.run(
    "MultiProc",
    plugin_args={"n_procs": 4}
)

## Method 2

In [7]:
import os
import nipype.interfaces.spm as spm

In [12]:
seg = spm.Segment(data=["/mnt/HDD/Data/test/data/subject_01.nii"])
# seg.inputs.data = os.path.commonpath("mnt/HDD/Data/test/data/subject_01.nii")
seg.run()

RuntimeError: Command:
matlab -nodesktop -nosplash -singleCompThread -r "addpath('/home/kaihulme/Projects/medical-gan-local/notebooks');pyscript_segment;exit"
Standard output:

                            < M A T L A B (R) >
                  Copyright 1984-2021 The MathWorks, Inc.
                  R2021a (9.10.0.1602886) 64-bit (glnxa64)
                             February 17, 2021

 
To get started, type doc.
For product information, visit www.mathworks.com.
 
Executing pyscript_segment at 20-Mar-2021 17:28:28:
------------------------------------------------------------------------------------------------------------------
MATLAB Version: 9.10.0.1602886 (R2021a)
MATLAB License Number: 911639
Operating System: Linux 5.8.0-7642-generic #47~1614007149~20.04~82fb226-Ubuntu SMP Tue Feb 23 02:56:27 UTC  x86_64
Java Version: Java 1.8.0_202-b08 with Oracle Corporation Java HotSpot(TM) 64-Bit Server VM mixed mode
----------------------------------------------------------------------------